In [ ]:

import requests

headers = {
    'accept': '*/*',
    'content-type': 'application/json',
}

json_data = {
    'operationName': 'Performers',
    'variables': {
        'input': {
            'names': '',
            'is_favorite': True,
            'page': 1,
            'per_page': 25,
            'sort': 'NAME',
            'direction': 'ASC',
        },
    },
    'query': 'query Performers($input: PerformerQueryInput!) {\n  queryPerformers(input: $input) {\n    count\n    performers {\n      id\n      name\n      disambiguation\n      deleted\n      aliases\n      gender\n      birth_date\n      age\n      height\n      hair_color\n      eye_color\n      ethnicity\n      country\n      career_end_year\n      career_start_year\n      breast_type\n      waist_size\n      hip_size\n      band_size\n      cup_size\n      tattoos {\n        location\n        description\n        __typename\n      }\n      piercings {\n        location\n        description\n        __typename\n      }\n      urls {\n        ...URLFragment\n        __typename\n      }\n      images {\n        ...ImageFragment\n        __typename\n      }\n      is_favorite\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment URLFragment on URL {\n  url\n  site {\n    id\n    name\n    icon\n    __typename\n  }\n  __typename\n}\n\nfragment ImageFragment on Image {\n  id\n  url\n  width\n  height\n  __typename\n}',
}

response = requests.post('https://stashdb.org/graphql', cookies=cookies, headers=headers, json=json_data)
data = response.json()
ids = [x['id'] for x in data['data']['queryPerformers']['performers']]
ids

In [ ]:
tpdb_ids = {}

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/json',
    'origin': 'https://stashdb.org',
}

for id in ids:
    json_data = {
        'operationName': 'FullPerformer',
        'variables': {
            'id': id,
        },
        'query': 'query FullPerformer($id: ID!) {\n  findPerformer(id: $id) {\n    ...PerformerFragment\n    studios {\n      scene_count\n      studio {\n        id\n        name\n        parent {\n          id\n          name\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment URLFragment on URL {\n  url\n  site {\n    id\n    name\n    icon\n    __typename\n  }\n  __typename\n}\n\nfragment ImageFragment on Image {\n  id\n  url\n  width\n  height\n  __typename\n}\n\nfragment PerformerFragment on Performer {\n  id\n  name\n  disambiguation\n  deleted\n  aliases\n  gender\n  birth_date\n  age\n  height\n  hair_color\n  eye_color\n  ethnicity\n  country\n  career_end_year\n  career_start_year\n  breast_type\n  waist_size\n  hip_size\n  band_size\n  cup_size\n  tattoos {\n    location\n    description\n    __typename\n  }\n  piercings {\n    location\n    description\n    __typename\n  }\n  urls {\n    ...URLFragment\n    __typename\n  }\n  images {\n    ...ImageFragment\n    __typename\n  }\n  is_favorite\n  __typename\n}',
    }

    response = requests.post('https://stashdb.org/graphql', cookies=cookies, headers=headers, json=json_data)
    response.raise_for_status()
    performer_data = response.json()
    print(f"Performer: {performer_data}")

    try:
        porndb_url = next(url['url'] for url in performer_data['data']['findPerformer']['urls'] if url['site']['name'] == 'ThePornDB')
        response = requests.get(porndb_url, cookies=tpdb_cookies)
        response.raise_for_status()
        from bs4 import BeautifulSoup
        import json
        parsed_html = BeautifulSoup(response.text, 'html.parser')
        tpdb_performer_data = json.loads(parsed_html.find(id="app")['data-page'])['props']['performer']
        tpdb_ids[tpdb_performer_data['id']] = tpdb_performer_data['slug']
    except Exception as e:
        print("tpdb url not found, trying to scrape using name...")
        response = requests.get("https://theporndb.net/performers?orderBy=recently_created&page=1&q=" + requests.utils.quote(performer_data['data']['findPerformer']['name']), cookies=tpdb_cookies)
        response.raise_for_status()
        from bs4 import BeautifulSoup
        import json
        parsed_html = BeautifulSoup(response.text, 'html.parser')
        data = json.loads(parsed_html.find(id="app")['data-page'])
        tpdb_performer_data = data['props']['performers']['data'][0]
        tpdb_ids[tpdb_performer_data['id']] = tpdb_performer_data['slug']



In [64]:
import requests

params = ''

for id, name in tpdb_ids.items():

    json_data = {
        'enableAutomaticAdd': True,
        'searchForMissingEpisodes': False,
        'shouldMonitor': 'specificEpisode',
        'siteMonitorType': 'all',
        'monitorNewItems': 'all',
        'qualityProfileId': 1,
        'listType': 'advanced',
        'listOrder': 5,
        'minRefreshInterval': '06:00:00',
        'fields': [
            {
                'name': 'performerId',
                'value': str(id),
            },
        ],
        'implementationName': 'TPDb Performer',
        'implementation': 'TPDbPerformer',
        'configContract': 'TPDbPerformerSettings',
        'infoLink': 'https://wiki.servarr.com/whisparr/supported#tpdbperformer',
        'tags': [],
        'name': f'{name} - {id}',
        'rootFolderPath': '/data/media/whisparr',
    }

    response = requests.post(
        'http://flab.local:6969/api/v3/importlist',
        params=params,
        cookies=whisparr_cookies,
        headers=whisparr_headers,
        json=json_data,
        verify=False,
    )


In [61]:
def create_tag(name):
    import requests
    json_data = {
        'label': name,
    }

    response = requests.post('http://flab.local:6969/api/v3/tag', cookies=whisparr_cookies, headers=whisparr_headers, json=json_data, verify=False)
    response.raise_for_status()
    return response.json()['id']
